# load and process the dataset

In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv("../data/tic-tac-toe.data", header=None)
data.columns = [
    "top-left", "top-middle", "top-right",
    "middle-left", "middle-middle", "middle-right",
    "bottom-left", "bottom-middle", "bottom-right",
    "outcome"
]

# Display the first few rows
print(data.head())

  top-left top-middle top-right middle-left middle-middle middle-right  \
0        x          x         x           x             o            o   
1        x          x         x           x             o            o   
2        x          x         x           x             o            o   
3        x          x         x           x             o            o   
4        x          x         x           x             o            o   

  bottom-left bottom-middle bottom-right   outcome  
0           x             o            o  positive  
1           o             x            o  positive  
2           o             o            x  positive  
3           o             b            b  positive  
4           b             o            b  positive  


In [3]:
# Encode the data
mapping = {'x': 1, 'o': -1, 'b': 0, 'positive': 1, 'negative': 0}
data = data.replace(mapping).infer_objects(copy=False)

# Separate features and target
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

print("Processed data:")
print(X[:5], y[:5])

Processed data:
[[ 1  1  1  1 -1 -1  1 -1 -1]
 [ 1  1  1  1 -1 -1 -1  1 -1]
 [ 1  1  1  1 -1 -1 -1 -1  1]
 [ 1  1  1  1 -1 -1 -1  0  0]
 [ 1  1  1  1 -1 -1  0 -1  0]] [1 1 1 1 1]
